<a href="https://colab.research.google.com/github/MarnixVos/Computer-Science/blob/main/CS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
#Import packages
import json
import pandas as pd
import numpy as np
import regex as re
import random

from random import shuffle
from pprint import pprint
from itertools import combinations
from sympy import isprime

In [ ]:
# grab JSON file
use_real = True
file_real = "/content/gdrive/MyDrive/CS/TVs-all-merged.json"
file_test = "/content/gdrive/MyDrive/CS/TVs-minimal.json"
# file_test = "test.json"

if(use_real):
    with open(file_real) as json_file:
        input = json.load(json_file)
else:
    with open(file_test) as json_file:
        input = json.load(json_file)

print("Data loaded!")

Data loaded!


In [ ]:
# Hyperparams
#k = 5  # shingle length
m = 20  # minhash dense vector length
b = 4  # amount of bands for LSH (make sure m is a multiple of b)
bootstraps = 10
sim_threshold = 0.3
obs = 1000
print("Hyper Params set")

Hyper Params set


In [ ]:
#-----------------------------------------------------------------------------------------------------------------#
#_----------------------------------------------------HELPER FUNCTIONS--------------------------------------------#
#-----------------------------------------------------------------------------------------------------------------#

# Extract data
def create_data_dict(data):
  dct = {}
  mapping_dict = {}
  counter = 0 
  for key in data:
      products = data[key]
      for i in range(0, len(products)):
          title = data[key][i].get('title')
          mapping_dict.update({counter : key + "|||" + data[key][i].get('shop')})
          dct.update({key + "|||" + data[key][i].get('shop'): title})
          counter +=1
  return dct, mapping_dict

def create_model_words(title):

  #Standardize inch values
  inch_values = ["Inch", 'inches', '\"', '-inch', ' inch']
  for inch_value in inch_values:
    title = title.replace(inch_value, 'inch')

  #Standardize Hz values
  hz_values = ["Hertz", 'hertz', 'Hz', 'HZ', '-hz']
  for hz_value in hz_values:
    title = title.replace(hz_value, 'hz')

  #Remove site titles
  sites = ['TheNerds.net', 'Newegg.com']
  for site in sites:
    title = title.replace(site, '')

  #All to lower case
  title = title.lower()
  

  model_words = re.findall("[a-zA-Z0-9\.]+", title)
  words_1 = [string for string in model_words if not string.isalpha()]
  words = [string for string in words_1 if not string.isnumeric()]

  return words

# Shingling function
def create_shingles(doc, k):
    shingles = set()
    length = len(doc)
    for index in range(length-k+1):
        shingle = doc[index:index+k]
        shingles.add(shingle)

    return shingles


# one hot encode shingle
def encode_one_hot(shingles, vocab):
    sparse_vec =  np.zeros(len(vocab))
    for shingle in shingles:
        idx = vocab.get(shingle)
        sparse_vec[idx] = 1
        # vocab_size = size of the vocab
    return sparse_vec

def hash_function(a,b,p,x):
  return (a + b*x)


# We have to save our hash functions and reuse them, so we have 20 different hash functions. Not 20x how ever many sparse vectors we have
# def create_hash_funcs(vocab_size, m):
#     hashes = np.zeros((m, vocab_size))
#     for i in range(m):
#         hash = np.random.permutation(len(vocab)) + 1
#         hashes[i, :] = hash.copy()    
#     return hashes.astype(int)


#We have to save our hash functions and reuse them, so we have 20 different hash functions. Not 20x how ever many sparse vectors we have
def create_hash_funcs(vocab_size, m):
    hashes = np.zeros((m, vocab_size))
    primes = [i for i in range(m,m*m) if isprime(i)]
    for i in range(m):
        a = random.randint(1, 100)
        b = random.randint(1, 100)
        p = np.repeat(random.choice(primes), vocab_size)
        f = lambda x: a+b*x
        base = np.arange(vocab_size)+1
        func = f(base)
        hash = np.mod(func, p)
        hashes[i, :] = hash.copy()    
    return hashes.astype(int)

# create signatures
# hash = a hash from create_hash_func
# sparse_vec = the original sparse vector
# vocab_size = size of the vocabulary


# create complete signature vector

def create_minhash_signature_vector(sparse_vec, vocab_size, m, hashes):
    idxs_to_check = np.nonzero(sparse_vec)[0].tolist()
    hash_values = hashes[:, idxs_to_check]
    signatures = np.min(hash_values, axis=1)
    return signatures

# calculate Jacard Similarity


def calculate_jacard_similarity(a, b):
    return len(a.intersection(b)) / len(a.union(b))


# split vector into bands
def create_bands(signature, b):
    r = int(len(signature)/b)
    bands = []
    for i in range(0, len(signature), r):
        val = str(signature[i: i+r])
        bands.append(val)
    return np.stack(bands)


#Get canidate pairs
def get_canidate_pairs(buckets): 
  canidates= []
  for key in buckets: 
    products_in_bucket = buckets[key]
    if len(products_in_bucket) > 1:
      canidates.extend(combinations(products_in_bucket,2))
  return set(canidates)




def generate_canidate_pairs(canidates):
  canidate_pairs = []
  for canidate_bucket in canidates:
    canidate_list = canidate_bucket.strip('][').split(', ')
    n = len(canidate_list)
    for i in range(n):
      for j in range(i+1, n):
        canidate_pairs.append([canidate_list[i],canidate_list[j]])

  return canidate_pairs

# find match between bands, only need 1 band to match to hash into the same bucket


def detect_equal_signature(sig1, sig2):
    bands1 = split_signature(sig1, b)
    bands2 = split_signature(sig2, b)
    for band_1, band_2 in zip(bands1, bands2):
        if band_1 == band_2:
            print("MATCH FOUND")
            break

def evaluate_pairs(pairs, mapping_dict,data): 
  #Evaluate pairs that we have marked as duplicates
  TP = []
  FP = []

  for pair in pairs: 
    i1 = int(pair[0])
    i2 = int(pair[1])
    if(i1 == i2):
      pass
    else:
      p1 = str.split(mapping_dict.get(i1),'|||')
      p2 = str.split(mapping_dict.get(i2),'|||')

      data1 = data.get(p1[0])
      data2 = data.get(p2[0])

      model1 = data.get(p1[0])[0]['modelID']
      model2 = data.get(p2[0])[0]['modelID']

      if (model1 == model2):
        TP.append(pair)
      else: 
        FP.append(pair)

  return [len(TP) , len(FP)]

def equal_tv_brands(title1, title2):
    #If the brands are recognized and not equal, we mark as not equal 
    tv_brands = ['philips', 'supersonic', 'sharp', 'samsung', "nec", 'tcl', 'toshiba', 'hisense', 'sony', 'lg', 'sanyo', 
                 'coby', 'panasonic', 'rca', 'sunbritetv', 'jvc', 'insignia', 'haier', 'optoma', 'vizio', 'westinghouse',
                 'sansui']
    brand1 = "."
    brand2 = "."

    #print(f"TV1: {str.lower(product_data1['title'])} TV2: {str.lower(product_data2['title'])}")
    for tv_brand in tv_brands: 
      
      if tv_brand in title1:
        brand1 = tv_brand
      if tv_brand in title2:
        brand2 = tv_brand
    
    if(brand1 is not brand2):
      if (brand1 is "." or brand2 is "."):
        return True
      else:
        return False
    else: 
      return True

def equal_resolutions(title1, title2):
  resolution1 = "."
  resolution2 = "."
  resolutions = ['720p', '1080p']
  for resolution in resolutions: 
    if resolution in title1:
      resultion1 = resolution
    if resolution in title2:
      resultion2 = resolution

  if(resolution1 is not resolution2):
      if (resolution1 is "." or resolution2 is "."):
        return True
      else:
        return False
  else: 
      return True

def equal_refresh_rates(title1, title2):
    refresh_rate1 = '.'
    refresh_rate2 = "."
    refresh_rates = ['60hz', '120hz', '600hz', '240hz']
    for refresh_rate in refresh_rates: 
      if refresh_rate in title1: 
        refresh_rate1 = refresh_rate
      if refresh_rate in title2: 
        refresh_rate2 = refresh_rate
    
    if(refresh_rate1 is not refresh_rate2):
      if (refresh_rate1 is "." or refresh_rate2 is "."):
        return True
      else:
        return False
    else: 
      return True


def standardize_product_title(title): 
    title = str.lower(title)
    #Standardize inch values
    inch_values = ["inch", 'inches', '\"', '-inch', ' inch']
    for inch_value in inch_values:
      title = title.replace(inch_value, 'inch')

    #Standardize Hz values
    hz_values = ['hertz', '-hz', ' hz']
    for hz_value in hz_values:
      title = title.replace(hz_value, 'hz')
    
    return title



def get_duplicates(canidates, mapping_dict, data, matrix):
  #Now we analyze the canidate pairs 
  duplicates = []
  for pair in canidates: 
    i1 = int(pair[0])
    i2 = int(pair[1])
  
    #check if its the exact same product, in that case dont mark as duplicate pair
    if(i1 == i2):
      pass
    else:  
      #Grab the product key and correspoding store
      p1 = str.split(mapping_dict.get(i1),'|||')
      p2 = str.split(mapping_dict.get(i2),'|||')

      #Get original data set for product 1 
      products1 = data.get(p1[0], )
      for product in products1: 
        if product['shop'] == p1[1]:
          product_data1 = product

      #Get original data set for product 12
      products2 = data.get(p2[0], )
      for product in products2: 
        if product['shop'] == p2[1]:
          product_data2 = product

      #Now we need to classify if, based on these two product sets, the models are indeed equal
      title1 = standardize_product_title(product_data1['title'])
      title2 = standardize_product_title(product_data2['title'])
      
      if(equal_tv_brands(title1, title2)):
        #Are the resolutions equal:
        if(equal_resolutions(title1, title2)):
          #Are the refresh rates equal
          if(equal_refresh_rates(title1, title2)):
            #Is the sim at least above the threshold: 
            idxs1 = np.nonzero(matrix[i1, :])[0].tolist()
            idxs2 = np.nonzero(matrix[i2, :])[0].tolist()
            sim = calculate_jacard_similarity(set(idxs1), set(idxs2))
            if (sim) > sim_threshold:
              duplicates.append(pair)
            


  return duplicates

#Get total amount of duplicate pairs in the data
def get_total_duplicate_pairs(data):
  total = 0 
  for key in data: 
    l = len(data[key])
    if l > 1: 
      for i in range(1,l):
        total += i
  return total

def get_total_amount_products(data): 
  total = 0
  max_comps =0 
  for key in data: 
    l = len(data[key])
    total += l 
  for i in range(1, total): 
    max_comps += i 


  return total, max_comps
#We now have canidate pairs, evaluate performance
def evaluate_performance(canidates, duplicates, data, mapping_dict):
  total_comps = len(canidates)
  total_dups = get_total_duplicate_pairs(data)
  TP, FN = evaluate_pairs(duplicates, mapping_dict, data)
  recall = TP / total_dups
  precision = TP / (TP + FN)
  PQ = TP / total_comps
  PC = TP / total_dups
  total_products, max_comps = get_total_amount_products(data)
  frac_of_comp = total_comps / max_comps
  F1 = 2*precision*recall / (precision + recall)
  F1_star =  2*PQ*PC / (PQ + PC)

  performance = {
    "F1": F1,
    "F1_star": F1_star,
    "PQ": PQ,
    "PC": PC,
    "Recall": recall,
    "Precision": precision,
    "Frac Comps": frac_of_comp,
    "Total products": total_products
  }

  return performance

In [ ]:
#-----------------------------------------------------------------------------------------------------------------#
#_-------------------------------------------------------MAIN FUNCTION--------------------------------------------#
#-----------------------------------------------------------------------------------------------------------------#



# This part extracts all the titles and adds them to a dict
# ISSUE: duplicate items get filed under the same key, but we want different keys here for comparisons >  SOLUTION use key + shop as unique identifier


def main_func(data):
  title_dict, mapping_dict = create_data_dict(data)
  #print("------ EXTRACTED ALL THE TITLES -------")

  # Now we create a list of all our model words and also create our vocab for one hot encoding
  model_words_dict = {}
  vocab = set()
  for key in title_dict:
      title = title_dict[key]
      model_words = create_model_words(title)
      model_words_dict.update({key: model_words})
      vocab = set.union(vocab, model_words)
  #print("------ CREATED MODELWORDS -------")
  # print("VOCAB: ", vocab)
  #pprint(model_words_dict)


  total_model_words = len(vocab)

  # CREATE THE MINHASH MATRIX
  # we need a dictionary for the vocab to get O(1) access time
  vocab_dict = {}

  i = 0
  for model_word in vocab:
      vocab_dict.update({model_word: i})
      i = i+1

  #Create one hot vector representation
  matrix = []
  for key in model_words_dict:
      model_words_set = model_words_dict[key]
      matrix.append(encode_one_hot(model_words_set, vocab_dict))
  #print("------ CREATED MATRIX SPARSE VEC REPRESENTATION -------")
  matrix = np.stack(matrix)
  #print(matrix.shape)


  # Create MinHash Matrix with signature of length m
  signature_matrix = []
  hashes = create_hash_funcs(total_model_words, m)
  for vector in matrix:
    signature_matrix.append(create_minhash_signature_vector(vector, total_model_words, m, hashes))

  #print("------ CREATED MINHASH DATAFRAME -------")
  signature_matrix = np.stack(signature_matrix)
  #print(signature_matrix, signature_matrix.shape)


  #Generate dictionary of buckets
  buckets = {}

  i = 0
  for signature in signature_matrix:
    bands = create_bands(signature, b)
    for band in bands: 
      if band in buckets: 
        buckets[band].append(i)
      else:
        buckets.update({band : [i]})

    i += 1

  #print("------ CREATED LSH BUCKETS -------")


  #get canidate pairs
  canidates = get_canidate_pairs(buckets)
  #print("------ CREATED CANIDATE PAIRS -------")


  duplicates = get_duplicates(canidates, mapping_dict, data, matrix)
  #print("------ DETERMINED DUPLICATE PAIRS -------")

  performance = evaluate_performance(canidates, duplicates, data, mapping_dict)
  return performance

In [ ]:
#-----------------------------------------------------------------------------------------------------------------#
#_-------------------------------------------------------BOOTSTRAPPING--------------------------------------------#
#-----------------------------------------------------------------------------------------------------------------#


performances = []
_ , main_mapping_dict = create_data_dict(input)
for z in range(1,bootstraps+1):
  print("Bootstrap ", z)
  bootstrap_data = {}
  for _ in range(obs):
    idx = random.randint(0,1623)
    product_name, product_store = str.split(main_mapping_dict.get(idx),'|||')
    products = input.get(product_name)
    for product in products: 
      if product['shop'] == product_store:
        product_dict = product
        product_key = product_dict['modelID']

    if product_key in bootstrap_data:
      bootstrap_data[product_key].append(product_dict)
    else: 
      bootstrap_data.update({product_key : [product_dict]})
  performance = main_func(bootstrap_data)
  performances.append(performance)

Bootstrap  1
Bootstrap  2
Bootstrap  3
Bootstrap  4
Bootstrap  5
Bootstrap  6
Bootstrap  7
Bootstrap  8
Bootstrap  9
Bootstrap  10


In [ ]:
df = pd.DataFrame(performances)
means = df.mean(axis=0)
print(means)

F1                   0.034952
F1_star              0.010671
PQ                   0.006581
PC                   0.028852
Recall               0.028852
Precision            0.044999
Frac Comps           0.004129
Total products    1000.000000
dtype: float64
